In [1]:
import tensorflow as tf
import math
# from datasets import fishes
from datasets import dataset_factory


In [16]:
slim = tf.contrib.slim

with tf.Graph().as_default():
    dataset = dataset_factory.get_dataset(
                "fishes", "validation", "tmp/fish/dataset")
    num_readers = 5
    batch_size = 4
    provider = slim.dataset_data_provider.DatasetDataProvider(
                        dataset,
                        num_readers=num_readers,
                        common_queue_capacity=20 * batch_size,
                        common_queue_min=10 * batch_size,
                        shuffle=True)
    image, label, bboxes, name = provider.get(['image', 'object/label', 'object/bbox', 'name'])
    images, labels, bboxes, name = tf.train.batch(
          [image, label, bboxes, name],
          batch_size=batch_size,
          num_threads=1,
          capacity=2 * batch_size,
          dynamic_pad=True)
    with tf.Session() as sess:
        with slim.queues.QueueRunners(sess):
            sess.run(tf.local_variables_initializer())
            np_images, np_bboxes, labels, name = sess.run([images, bboxes, labels, name])


In [17]:
np_images.shape

(4, 750, 1280, 3)

In [13]:
labels

array([[8, 0],
       [0, 0],
       [1, 0],
       [4, 4],
       [7, 0],
       [1, 0],
       [6, 0],
       [2, 0]])

In [15]:
np_bboxes

array([[[ 0.52483332,  0.28860939,  0.71479166,  0.48138282],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.77980757,  0.45624569,  0.82234251,  0.51701206],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.175     ,  0.41249999,  0.70694447,  0.75859374],
        [ 0.61250001,  0.66562498,  0.9847222 ,  0.890625  ]],

       [[ 0.18133333,  0.27109376,  0.53866667,  0.6640625 ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.49448863,  0.40294594,  0.76240551,  0.49507445],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.59444445,  0.27656251,  0.69722223,  0.45468751],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.40554416,  0.22890624,  0.78131419,  0.36328125],
        [ 0.        ,  0.        ,  0.        ,  0.     